In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./data/test.tsv
./data/dev.tsv
./data/train.tsv


In [2]:
# !git clone https://github.com/shahrukhx01/fine-tune-bert.git

In [3]:
# %cd fine-tune-bert/

In [4]:
import torch
from model import BERTClassifier
from config import BertOptimConfig
from train import train_model
from data_loader import QuestionsDataLoader

    

2023-01-10 15:03:53.420275: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-10 15:03:53.761492: E tensorflow/tsl/lib/monitoring/collection_registry.cc:81] Cannot register 2 metrics with the same name: /tensorflow/core/bfc_allocator_delay
2023-01-10 15:03:54.370559: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libnvinfer.so.8'; dlerror: libnvinfer.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/lb/cuda/lib64:/usr/local/cuda-11.2/lib64
2023-01-10 15:03:54.370648: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libnvinfer_plugin.so.8'; dlerror: libnvinfer_plugin.so.8: cannot open shared object file: No such

In [5]:
epochs = 2
num_labels = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

"""
data_path = {
    'train': '/kaggle/input/quora-question-keyword-pairs/train.tsv',
    'dev': '/kaggle/input/quora-question-keyword-pairs/dev.tsv',
    'test': '/kaggle/input/quora-question-keyword-pairs/test.tsv'
}
data_loaders = QuestionsDataLoader(data_path, batch_size=8)
model = BERTClassifier(num_labels=num_labels).get_model()
optim_config = BertOptimConfig(model=model, train_dataloader=data_loaders.train_dataloader, epochs=epochs)

## execute the training routine
model = train_model(model=model, 
            optimizer=optim_config.optimizer, 
            scheduler=optim_config.scheduler, 
            train_dataloader=data_loaders.train_dataloader, 
            validation_dataloader=data_loaders.validation_dataloader, 
            epochs=epochs, device=device)
"""
## test model performance on unseen test set
##eval_model(model=model, test_dataloader=data_loaders.test_dataloader, device=device)


"\ndata_path = {\n    'train': '/kaggle/input/quora-question-keyword-pairs/train.tsv',\n    'dev': '/kaggle/input/quora-question-keyword-pairs/dev.tsv',\n    'test': '/kaggle/input/quora-question-keyword-pairs/test.tsv'\n}\ndata_loaders = QuestionsDataLoader(data_path, batch_size=8)\nmodel = BERTClassifier(num_labels=num_labels).get_model()\noptim_config = BertOptimConfig(model=model, train_dataloader=data_loaders.train_dataloader, epochs=epochs)\n\n## execute the training routine\nmodel = train_model(model=model, \n            optimizer=optim_config.optimizer, \n            scheduler=optim_config.scheduler, \n            train_dataloader=data_loaders.train_dataloader, \n            validation_dataloader=data_loaders.validation_dataloader, \n            epochs=epochs, device=device)\n"

In [6]:
# !wget https://get.station307.com/gIfe63unMw4/query_classifier.pickle

In [7]:
import pickle
"""with open('query_classifier.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)"""
with open('query_classifier.pickle', 'rb') as handle:
    model = pickle.load(handle)

In [9]:
import pandas as pd
import torch
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from tqdm import tqdm
import logging

## setting the threshold of logger to INFO
logging.basicConfig(filename='data_loader.log', level=logging.INFO)

## creating an object
logger = logging.getLogger()
  


class QuestionsData:
    def __init__(self, data_path, start_offset=0, end_offset=1000, max_sequence_length=512):
        """
        Load dataset and bert tokenizer
        """
        ## load data into memory
        self.test_df = pd.read_csv(data_path['test'], sep='\t')[start_offset:end_offset]
        ## set max sequence length for model
        self.max_sequence_length = max_sequence_length
        ## get bert tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-mini', do_lower_case=True)
    
    def train_val_test_split(self):
        """
        Separate out labels and texts
        """
        test_texts = self.test_df['query'].values
        test_labels = self.test_df.target.values
        
        return test_texts, test_labels
    
    def preprocess(self, texts):
        """
        Add bert token (CLS and SEP) tokens to each sequence pre-tokenization
        """
        ## separate labels and texts before preprocessing
        # Adding CLS and SEP tokens at the beginning and end of each sequence for BERT
        texts_processed = ["[CLS] " + str(sequence) + " [SEP]" for sequence in texts]
        return texts_processed
        
    def tokenize(self, texts):
        """
        Use bert tokenizer to tokenize each sequence and post-process 
        by padding or truncating to a fixed length
        """
        ## tokenize sequence
        tokenized_texts = [self.tokenizer.tokenize(text) for text in (texts)]
        
        ## convert tokens to ids
        text_ids = [self.tokenizer.convert_tokens_to_ids(x) for x in (tokenized_texts)]

        ## pad our text tokens for each sequence
        text_ids_post_processed = pad_sequences(text_ids, 
                                       maxlen=self.max_sequence_length, 
                                       dtype="long", 
                                       truncating="post", 
                                       padding="post") 
        return text_ids_post_processed

    def create_attention_mask(self, text_ids):
        """
        Add attention mask for padding tokens
        """
        attention_masks = []
        # create a mask of 1s for each token followed by 0s for padding
        for seq in (text_ids):
            seq_mask = [float(i>0) for i in seq]
            attention_masks.append(seq_mask)
        return attention_masks

    def process_texts(self):
        """
        Apply preprocessing and tokenization pipeline of texts
        """
        ## perform the split
        test_texts, test_labels = self.train_val_test_split()

        ## preprocess train, val, test texts
        test_texts_processed = self.preprocess(test_texts)
        
        del test_texts
        
        ## preprocess train, val, test texts
        test_ids = self.tokenize(test_texts_processed)

       
        del test_texts_processed 
        
        ## create masks for train, val, test texts
        test_masks = self.create_attention_mask(test_ids)
        return (
                test_ids,
                test_masks,
                test_labels
                )

    
    def text_to_tensors(self):
        """
        Converting all the data into torch tensors
        """
        test_ids, \
        test_masks, \
        test_labels = self.process_texts()

        
        ## convert inputs, masks and labels to torch tensors
       
        self.test_inputs = torch.tensor(test_ids)
        self.test_labels = torch.tensor(test_labels)
        self.test_masks = torch.tensor(test_masks)



In [10]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

class QuestionsDataLoader:

    def __init__(self, data, data_file, batch_size=8):
        self.spam_data = data
        self.batch_size = batch_size
        self.create_loaders()
    
    def create_loaders(self):
        """
        Create Torch dataloaders for data splits
        """
        self.spam_data.text_to_tensors()
        
        
        test_data = TensorDataset(self.spam_data.test_inputs, 
                                        self.spam_data.test_masks, 
                                        self.spam_data.test_labels)
        test_sampler = SequentialSampler(test_data)
        self.test_dataloader = DataLoader(test_data, 
                                                sampler=test_sampler, 
                                                batch_size=self.batch_size)
        

In [11]:
import torch
from utils import flat_accuracy

def eval_model(model, test_dataloader, device):
    ## tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    ## evaluate data for one epoch
    for batch in test_dataloader:
        ## add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        ## unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        ## avoiding model's computation and storage of gradients -> saving memory and speeding up validation
        with torch.no_grad():
            # forward pass, calculate logit predictions
            logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        
        ## move logits and labels to CPU
        logits = logits['logits'].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    return eval_accuracy/nb_eval_steps

In [14]:
if __name__=='__main__':
    data_path = {
    'train': './data/train.tsv',
    'dev': './data/dev.tsv',
    'test': './data/test.tsv'
    }
    outer_batch_size = 10000
    accuracy = []
    for i in tqdm(range(outer_batch_size, 1557858, outer_batch_size)):
        ## batching test set
        data = QuestionsData(data_path, start_offset=i-outer_batch_size, end_offset=i)
        quest_loader = QuestionsDataLoader(data, data_path)
        quest_loader.create_loaders()
        # accuracy.append(eval_model(model=model, test_dataloader=quest_loader.test_dataloader, device=device))

  1%|          | 1/155 [00:15<38:59, 15.19s/it]


KeyboardInterrupt: 

In [ ]:
import numpy as np
np.mean(accuracy)